In [1]:
import feedWebGL2.feedback as fd
fd.widen_notebook()

In [2]:
shader_GLSL_code = """#version 300 es

// Shift all input_vectors by a single shared shift_vector.
// This shader program runs in parallel for all input vectors.

// Input vertices bind to this variable in parallel.
in vec3 input_vertex;

// vector shift value shared across all processing,
uniform vec3 shift_vector;

// feedback: output shifted vertex
out vec3 output_vertex;

void main() {
    output_vertex = input_vertex + shift_vector;
}
"""

feedback_program = fd.FeedbackProgram(
    program = fd.Program(
        vertex_shader = shader_GLSL_code,
        feedbacks = fd.Feedbacks(
            output_vertex = fd.Feedback(num_components=3),
        ),
    ),
    runner = fd.Runner(
        vertices_per_instance = len(vectors),
        uniforms = fd.Uniforms(
            shift_vector = fd.Uniform(default_value = [-1, 0, 1]) #
        ),
        inputs = fd.Inputs(
            input_vertex = fd.Input(  # Bind input_vertices buffer vectors to input_vertex variable
                num_components = 3,
                from_buffer = fd.BufferLocation(
                    name = "input_vertices", # start at the beginning, don't skip any values...
                )
            ),
        ),
    ),
    context = fd.Context(
        buffers = fd.Buffers(
            input_vertices = fd.Buffer(
                vectors = [
                    [1, 2, 3],  # values bound to input_vertex above in parallel
                    [2, 3, 4],
                    [3, 4, 5],
                ]
,
            ),
        ),
    ),
)

# display the widget and debugging information
feedback_program.debugging_display()

In [3]:
feedback_program.run()

In [4]:
feedback_program.get_feedback("output_vertex")

[[0, 2, 4], [1, 3, 5], [2, 4, 6]]